# EV長時間放置クラスタ予測 Notebook
要件に沿ってhashvinごとにデータ前処理・学習・評価を実施します。

## 0. セットアップ
依存パッケージの読み込みとパス設定を行います。

In [ ]:
from pathlib import Path
import sys
import json
import pandas as pd

# パス設定（このNotebookは multiclass_train ディレクトリで実行）
PROJECT_DIR = Path.cwd()
if str(PROJECT_DIR.parent) not in sys.path:
    sys.path.append(str(PROJECT_DIR.parent))

from multiclass_train.pipeline import PipelineConfig, HashvinProcessor
from multiclass_train.trainer import train_and_evaluate

## 1. 設定
パイプラインとAutoGluonの主要パラメータを定義します。

In [ ]:
pipeline_config = PipelineConfig(
    head_k=10,
    enable_station_type=False,  # TODO: ステーション種別利用時は True に設定
)

autogluon_presets = 'medium_quality_faster_train'
autogluon_time_limit = 300  # 秒。データ量に応じて調整
eval_thresholds = (0.5, 0.7, 0.9)

## 2. データ読込
CSVを読み込み、対象のhashvin一覧を確認します。

In [ ]:
SESSIONS_PATH = PROJECT_DIR / "ev_sessions_1.csv"
CHARGE_PATH = PROJECT_DIR / "ev_charge_to_long_inactive.csv"

charge_df = pd.read_csv(CHARGE_PATH)
sessions_df = pd.read_csv(SESSIONS_PATH)

hashvin_list = sorted(charge_df["hashvin"].unique())
target_hashvins = hashvin_list  # 一部だけ試す場合はリストを編集
print(f"hashvin count: {len(hashvin_list)}")
hashvin_list

## 3. 学習・評価実行
hashvinごとにパイプラインを実行し、結果をresult/以下へ保存します。

In [ ]:
RESULT_ROOT = PROJECT_DIR / "result"
RESULT_ROOT.mkdir(parents=True, exist_ok=True)

metrics_summary = []
for hashvin in target_hashvins:
    print(f"=== Processing {hashvin} ===")
    charge_subset = charge_df[charge_df["hashvin"] == hashvin].copy()
    sessions_subset = sessions_df[sessions_df["hashvin"] == hashvin].copy()

    processor = HashvinProcessor(hashvin, charge_subset, sessions_subset, pipeline_config)
    result = processor.build_features()

    hashvin_dir = RESULT_ROOT / hashvin
    metrics = train_and_evaluate(
        result,
        hashvin_dir,
        enable_station_type=pipeline_config.enable_station_type,
        autogluon_presets=autogluon_presets,
        autogluon_time_limit=autogluon_time_limit,
        eval_thresholds=eval_thresholds,
    )
    metrics_summary.append(metrics)

if metrics_summary:
    summary_path_json = RESULT_ROOT / "metrics_summary.json"
    summary_path_csv = RESULT_ROOT / "metrics_summary.csv"
    summary_path_json.write_text(json.dumps(metrics_summary, indent=2, ensure_ascii=False), encoding="utf-8")
    pd.DataFrame(metrics_summary).to_csv(summary_path_csv, index=False)
    display(pd.DataFrame(metrics_summary))
else:
    print("No hashvin processed. Check target_hashvins list.")

## 4. 出力物
- `result/<hashvin>/`: AutoGluonモデル、特徴量テーブル、評価結果、予測結果。
- `result/metrics_summary.json|csv`: hashvinごとの指標サマリ。